# Assignment 3
## Dog Breeds
## Arsenii Belyakov
### 450835

### uploading used libraries 

In [1]:
import os
import sys
import numpy as np
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from keras_applications.resnet import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

## Load data (wrong version, idk why)

In [2]:
dataList = os.listdir("../stanford-dogs-dataset/images/Images/") # os.listdir() method in python is used to get the list of all files and directories in the specified directory.
numOfClasses = len(dataList)
usedClasses = 3
imageCounter = 0
for dataClass in dataList:
    imageCounter+=len(os.listdir("../stanford-dogs-dataset/images/Images/"
                     .format(dataClass))) 
print(imageCounter)
print(numOfClasses)



14400
120


## Load data 

In [3]:
# os.listdir() method in python is used to get the list of all files and directories in the specified directory.
breed_list = os.listdir("../stanford-dogs-dataset/images/Images/")
usedBreeds = 3

num_classes = len(breed_list)
print("{} breeds".format(num_classes))

n_total_images = 0
for breed in breed_list:
    n_total_images += len(os.listdir("../stanford-dogs-dataset/images/Images/{}".format(breed)))
print("{} images".format(n_total_images))

120 breeds
20580 images


## Split data

Creating directroies and subdirectories for data

In [4]:
os.makedirs('data',exist_ok=True) #Like mkdir(), but makes all intermediate-level directories needed to contain the leaf directory.
os.makedirs('data/Train',exist_ok= True)
os.makedirs('data/Validate',exist_ok= True) 


Creating subbdirectories for breeds, which will be used

In [5]:
usedClasses = 1

for breed in breed_list:
    os.makedirs('data/Train/'+breed,exist_ok= True)
    os.makedirs('data/Validate/'+breed,exist_ok= True)
    if usedClasses >= usedBreeds: #reached limit of used breads
        break
    usedClasses+=1

    Save data into folders
cpt - Conditional probability table

In [6]:
validationToTraining = .1
breed_in_use = 1

for breed in os.listdir('data/Train/'):

    # walk() generates the file names in a directory tree by walking the tree either top-down or bottom-up.
    cpt = sum([len(files) for root, directory, files in os.walk('stanford-dogs-dataset/images/Images/{}/'.format(breed))])
    validation = (int)(cpt*validationToTraining)
    index = 0
    
    for file in os.listdir('../stanford-dogs-dataset/annotations/Annotation/{}'.format(breed)):
        
        img = Image.open('../stanford-dogs-dataset/images/Images/{}/{}.jpg'.format(breed, file))
        img = img.convert('RGB')
        
        if index<validation: #to make propper split
            img.save('data/Validate/' + breed + '/' + file + '.jpg')
        
        else:
            img.save('data/Train/' + breed + '/' + file + '.jpg')
        index +=1
        
    if breed_in_use == usedBreeds: #check if we are still in our limit
        break
        
    breed_in_use = breed_in_use+1
            

print(len(os.listdir('data/Train')))
print(len(os.listdir('data/Validate')))
            

3
3


# Step 1
## Build an input pipeline using the tf.data.Dataset API and/or keras.preprocessing.image API that loads the images, resizes them to the same resolution and applies some basic data augmentation like translation, rotation, etc. It should also prefetch the images